In [1]:
import numpy as np
import pandas as pd
import glob

/home/jlescutmuller/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
df = pd.read_pickle('./data/tweets_raw/v3/df.pkl')
print(df['id'].min())
print(df['id'].max())
print(df.shape)

1412369115575160836
1419276389878882308
(26895, 6)


# Bert Topic example (https://github.com/MaartenGr/BERTopic)

In [ ]:
# from bertopic import BERTopic
# from sklearn.datasets import fetch_20newsgroups
 
# docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

# topic_model = BERTopic()
# topics, _ = topic_model.fit_transform(docs)

# Manual Labeling before Semi-supervised learning :

In [ ]:
manual_label_conf = {
    
    ##############################
    # Industrial Applications :
    ##############################
    "application : industry (general)":{"industry":1, "enterprise":1, "business":1},
    "application : covid19":{"covid":20,"coronavirus":20,"delta variant":10,"delta": 3,"vaccine":1},
    "application : cybersecurity":{
        "cybersecurity": 10, "cyber-security": 10, "malware":5,
        "security":1, "cyber":1, "hack":1},
    "application : robotics":{"robotic":8,"robot":3},
    "application : finance":{"fintech":10,"financ":4,"bank":4,"economic":4},
    "application : climatology":{
        "weather forecast":10,"climato": 9, 
        "climate":4,"weather":4,"wind":3,"temperature":3,"atmosphere":3,"meteo":3},
    "application : healthcare":{
        "healthcare":10,
        "clinical":5,"medical":4,"medicine":4,"patients":3,
        "doctor":2, # can be a phd...
        "surgical":2,"surgery":2,"care":1},
    "application : marketing":{"marketing":5,"customer analy":5,"customers analy":5,"crm":5},
    "application : astronomy":{
        "astronomy":8,"universe":8,"star":1 # can be a rockstar..
        },
    "application : nuclear":{"nuclear":8,"nitrogen":8,"atomic":5},
    "application : cooking":{"cooking":5,"recipe":4,"food":5},
    "application : media & politics":{
        "trump":10,"biden":10,"federal":10,"media":6,"election":6,"president":6,"liberal":6,
        "politic":5,"conservat":4,"news":4,"positive":1,"negative":1},
    "afghanistan":{"afghanistan":5,"taliban":5},
    
    ##############################
    # Technics :
    ##############################
    "cloud technologies":{
        "amazon web service":6," aws ":6, "google cloud": 6, " gcp ":4,"azure":5,"cloud":4},
    "implementation & tools":{
         "keras":6,"pytorch":6,"tensorflow":6,"sklearn":4,
        "spark":4,"pandas":3,"python":2,"rstudio": 2," r ": 2,
        "matlab":2,"matplotlib":2," sas ":2,
        "javascript":2,"framework":1,
        'language': 2, 'code': 1, 'coding': 1},
    "maths":{
        "matrices":5,"matrix":5,'gradient': 5,
        'derivative': 4,'laplace':4,'lagrange':4,'poisson':4,'distribution': 2,
        "product":1, # Can be a product that you buy...
   },
   "devops":{"devops":5,"mlops":5,"dataops":5,"docker":5,"deploy":5},
    
    ##############################
    # Data-Science branch :
    ##############################
    "dataviz":{"dataviz":3," plot":2, "dashboard":2, "vizualize":1},
    "deep learning":{
        "neural network":10," deep learn": 9," deep ":1," rnn ":7," lstm ":6,
        "neural":5,"convolutional":5,"transformer":5},
    "machine learning":{" ml ":1,"machine learning":1},
    "AI":{"artificial intelligence":1," ai ":1, 'artificialintelligence': 1},
    
    ##############################
    # Academic/Training
    ##############################
    "university/school":{
        "scholarship":8," phd ":8,"universit":5,"school":5,"student":5,
        "academic":3,"fellowship":3, "professor":3,
        "graduate":2,"degree":2,'course':2,'diploma':2,'cursus':2,"certificat":2,
        "skill":1,'apply':1,"lecture": 1},
   "training":{
       "tuto":5,"handson":3,"hands on":3,
       "learn":2,"bootcamp":2,
       "training":1,"skill":1,
       'machine learn': -2 # Because of 'learn' up above
   },
   "book":{"book":5,"mustread":3,"must-read":3},
    
   ##############################
   # Others :
   ##############################
    "event":{
        "webinar":10,"conference":10,"tedx":5,"symposium":5,"virtual session":5,
        "virtual": 1,"session": 1,
        "speaker":4,"hackathon":4,"meetup":2,"event":2,
        "speech":1,"talk":1,"congres":1,"elevator pitch":1,"lecture":1},
    "recruiting":{
        "recruit":15, " hire":15, " hiring":15, "career":8, "resume":8, "cover letter": 8,
        "salaries":4, "high pay":4, "interview":3, "job":2, "candidate":2, 'role': 2,
        "intern ":2, "application":1, "profile":1, "skill":1}
}

In [ ]:
def get_topic_weights(text) :
    text = text.lower()  
    def _count(text, topic_name, topic_conf) :
        return sum([weight for (kw,weight) in topic_conf.items() if kw in text])
    w = {topic : _count(text, topic, conf) for (topic, conf) in manual_label_conf.items()}
    return w
df['topic_weights'] = df['text'].map(get_topic_weights)


def sorting_weights(w) :
    # Re-ordering by descending weights :
    keys = np.array(list(w.keys()))
    vals = np.array(list(w.values()))
    argsort = np.argsort(-vals)
    return {keys[i]:vals[i] for i in argsort}
    
df['topic_weights_sorted'] = df['topic_weights'].map(sorting_weights)

In [ ]:
df['label']    = df['topic_weights_sorted'].map(lambda w: None if sum(w.values()) == 0 else list(w.keys())[0])
df['label_w']  = df['topic_weights_sorted'].map(lambda w: None if sum(w.values()) == 0 else list(w.values())[0])
df['label2_w'] = df['topic_weights_sorted'].map(lambda w: None if sum(w.values()) == 0 else list(w.values())[1])

In [ ]:
px.scatter(df, x='label_w', y='label2_w')

In [ ]:
sum(df['label'].isnull())

In [ ]:
df.shape

# Sentence Embedding using Neural Networks (Transformers)

In [ ]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [ ]:
X_embedded = model.encode(df['text'].values)

# Try a t-SNE for vizualization :

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# X_label = np.vstack(df['topic_weights'].map(lambda w: np.array(list(w.values()))).values)
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_label = scaler.fit_transform(X_label)

# Tweeking of space :

In [ ]:
X_embedded_2 = X_embedded.copy()

In [ ]:
# SIZING DOWN clusters :
conf_individuals = {
     'application : industry (general)': 0.,
     'application : covid19': 0.2,
     'application : cybersecurity': 0.4,
     'application : robotics': 0.4,
     'application : finance': 0.2,
     'application : climatology': 0.2,
     'application : healthcare': 0.2,
     'application : marketing': 0.2,
     'application : astronomy': 0.4,
     'application : nuclear': 0.4,
     'application : cooking': 0.2,
     'application : media & politics': 0.6,
     'afghanistan': 0.4,
     'cloud technologies': 0.4,
     'implementation & tools': 0.4,
     'maths': 0.4,
     'devops': 0.4,
     'dataviz': 0.4,
     'deep learning': 0.,
     'machine learning': 0.,
     'AI': 0.,
     'university/school': 0.45,
     'training': 0.45,
     'book': 0.4,
     'event': 0.4,
     'recruiting': 0.45}

for topic, weight in conf_individuals.items() :
    mask = (df['label']==topic).values
    X_subset = X_embedded_2[mask,:]
    average_coordinates = X_embedded_2[mask,:].mean(axis=0)
    X_embedded_2[mask,:] = (1-weight)*X_embedded_2[mask,:] + weight*average_coordinates

In [ ]:
# MOVING SOME CLUSTER :
conf_groups = {
   "applications_stay_together":{
      "weight":.4,
      "clusters_to_move":[
         "application : covid19",
         "application : cybersecurity",
         "application : robotics",
         "application : finance",
         "application : climatology",
         "application : healthcare",
         "application : marketing",
         "application : astronomy",
         "application : nuclear",
         "application : cooking",
         "application : media & politics",
         "application : industry (general)",
      ]
   },
    
   "applications_average":{
      "weight":.8,
      "clusters_to_move":["application : industry (general)"],
      "target":[
         "application : covid19",
         "application : cybersecurity",
         "application : robotics",
         "application : finance",
         "application : climatology",
         "application : healthcare",
         "application : marketing",
         "application : astronomy",
         "application : nuclear",
         "application : cooking",
         "application : media & politics",
      ]
   },
    
    "health_application_should_stay_together":{
      "weight":.4,
      "clusters_to_move":[
          "application : covid19",
          "application : healthcare",
      ]
   },
#    "ds_fields":{
#       "weight":.3,
#       "similar_topics":[
#          "AI",
#          "machine learning",
#          "deep learning"
#       ]
#    },
   "technos":{
      "weight":.1,
      "clusters_to_move":[
         "cloud technologies",
         "implementation & tools",
         "devops"
      ]
   },
   "learning":{
      "weight":.4,
      "clusters_to_move":[
         "university/school",
         "training"
      ]
   }
}

for conf in conf_groups.values() :
    clusters_to_move = conf['clusters_to_move']
    weight = conf['weight']
    # By default : Move clusters closer to each others :
    target = conf['target'] if 'target' in conf.keys() else clusters_to_move
    
    target_mask = df['label'].isin(target).values
    target_avg_coordinates = X_embedded_2[target_mask,:].mean(axis=0)
    for clust in clusters_to_move :
        clust_mask = (df['label'] == clust).values
        clust_avg_coordinates = X_embedded_2[clust_mask,:].mean(axis=0)
        moving_vector = weight * (target_avg_coordinates - clust_avg_coordinates)
        X_embedded_2[clust_mask,:] += moving_vector

In [ ]:
# model = TSNE(n_components=3)
# X_tsne = model.fit_transform(X_train)
# df['tsne_0'] = X_tsne[:,0]
# df['tsne_1'] = X_tsne[:,1]
# df['tsne_2'] = X_tsne[:,2]
# fig = px.scatter_3d(df.loc[~df['label'].isnull()], x='tsne_0', y='tsne_1', z='tsne_2', 
#                     color='label')
# fig.update_traces(marker=dict(size=1))
# fig.show()

# Try a UMAP for vizualization :

In [ ]:
import umap
reducer = umap.UMAP(n_components=3)
X_reduced = reducer.fit_transform(X_embedded_2)
df['dim_0'] = X_reduced[:,0]
df['dim_1'] = X_reduced[:,1]
df['dim_2'] = X_reduced[:,2]

In [ ]:
# from plotly.express.colors import find_intermediate_color 
# PURPLE = (100.,  0.,  100.)
# BLUE   = (0.,    0.,  200.)
# GREEN  = (0.,    200., 0.)
# YELLOW = (100.,  100., 0.)
# ORANGE = (150.,  50., 0.)
# RED    = (200.,  0., 0.)

# color_discrete_map={
#     # GRAYS
#     'cloud technologies':     (125., 125., 125.),
#     'implementation & tools': (75.,  75.,  75.),
#     'devops' :                (25.,  25.,  25.),
    
#     # PURPLE & BLUES
#     'recruiting':          find_intermediate_color(PURPLE,BLUE,.25),
#     'training':            find_intermediate_color(PURPLE,BLUE,.50),
#     'book':                find_intermediate_color(PURPLE,BLUE,.75),
#     'university/school':   BLUE,
#     'event':               find_intermediate_color(BLUE,GREEN,.25),
     
#      # GREENS & YELLOWS
#     'AI':                  find_intermediate_color(BLUE,GREEN,.50),
#     'deep learning':       GREEN,
#     'machine learning':    find_intermediate_color(GREEN, YELLOW, .60),
#     'dataviz':             YELLOW,
#     'maths':               find_intermediate_color(YELLOW,ORANGE,.15),
    
    
#      # ORANGES & REDS
#      # Red-Purple / Red / Red-Orange / Orange
#     'application : media & politics':    find_intermediate_color(YELLOW,ORANGE,.50),
#     'application : marketing':           find_intermediate_color(YELLOW,ORANGE,.75),
#     'application : finance':             ORANGE,
#     'application : healthcare':          find_intermediate_color(ORANGE,RED,1/5),
#     'application : covid19':             find_intermediate_color(ORANGE,RED,2/5),
#     'application : cooking':             find_intermediate_color(ORANGE,RED,3/5),
#     'application : climatology':         find_intermediate_color(RED,PURPLE,4/5),
#     'application : industry (general)':  RED,
#     'application : astronomy':           find_intermediate_color(RED,PURPLE,1/6),
#     'application : nuclear':             find_intermediate_color(RED,PURPLE,2/6),
#     'application : cybersecurity':       find_intermediate_color(RED,PURPLE,3/6),
#     'application : robotics':            find_intermediate_color(RED,PURPLE,4/6),
#     'afghanistan':                       find_intermediate_color(RED,PURPLE,5/6),
# }
# color_discrete_map = {k:px.colors.label_rgb(v) for k,v in color_discrete_map.items()}

In [ ]:
fig = px.scatter_3d(df.loc[~df['label'].isnull()], x='dim_0', y='dim_1', z='dim_2', 
                    color='label', hover_name='text'
#                     color_discrete_map=color_discrete_map
                   )
fig.update_traces(marker=dict(size=1))
fig.show()

# Generalization with Classification

# Bert Topic applied to Twitter

In [ ]:
from bertopic import BERTopic
filt = df['label'].isnull()

topic_model = BERTopic() # int(sum(filt)/30)
topics, _ = topic_model.fit_transform(df.loc[filt, 'text'].values)
df['topic'] = None
df.loc[filt, 'topic'] = topics

In [ ]:
topic_model.get_topic_info().iloc[:10]

In [ ]:
topic_model.visualize_topics()

In [ ]:
# manual_overwrite = {
#     '@teqipf is delivering' : 'university/school',
#     'programming skills for data science (the r book)': 'book',
#     'top 10 data science coding schools': 'university/school',
#     'data science career': 'recruiting',
#     'machine learning tutorial': 'hands-on training',
#     'datascience certificates': 'online courses',
#     'top 10 in-demand tech skills': 'training',
#     'top 10 in-demand #techskills': 'training',
#     'azure ml studio': 'cloud technologies',
#     'vgpu': 'deep learning',
#     'kickstart your career': 'recruiting',
#     'postgrad conversion courses': 'university/school',
#     'in partnership with @awscloud': 'cloud technologies',
#     'businesses need to start upskilling': 'traininng',
#     'now hiring': 'recruiting',
#     'python vs r': 'implementation & tools',
#     'python programming': 'implementation & tools',
# }

In [ ]:
df['n_labels'] = df['manual_labels'].apply(lambda t: 0 if len(t) == 0 else len(t.split(',')))

In [ ]:
df = df.sort_values('n_labels', ascending=False)

In [ ]:
df.groupby('n_labels').size()

In [ ]:
df

In [ ]:
df['text'].values

In [ ]:
def manual_overwrite(text) :
    text = text.lower()
    for k,v in manual_overwrite.items() :
        if k in text :
            return v
df['manual_labels'] = df['text'].map(manual_labels)